import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=480
cls_num=5
btch=20
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=Haptics(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

first_train_shape= (155, 1093)
first_test_shape= (308, 1093)
classes_quantity= 5
tr_lbls=	 {1, 2, 3, 4, 5}
Count_labels= [18 34 34 36 33]
max(train_feature_Altitude)= 5.0
min(train_feature_Altitude)= -11.147
first_train_sample=
 [ 5.      -1.0475   0.54834 ...  0.89676 -2.6913  -8.1963 ]
1 18	2 34	3 34	4 36	5 33	

In [3]:
np.shape(ecg)

(155, 481)

In [4]:
ecg[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5.])

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=rate                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 18	2 34	3 34	4 36	5 33	
 max = 36


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (1,cls_num+1):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (18, 481)
Up to class  2 train shape =  (52, 481)
Up to class  3 train shape =  (86, 481)
Up to class  4 train shape =  (122, 481)
Up to class  5 train shape =  (155, 481)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[ 0 18 34 34 36 33]


In [8]:
xtrain[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5.])

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,cls_num+1):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (1,cls_num+1):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  4.597467569572298
min magnitude class 1  =  -13.17299850668317
after normalizing >>
max magnitude class 1  =  1.2596009116229263
min magnitude class 1  =  -1.8343638646323206

 cls 2  >> 
max magnitude class 2  =  2.7396485902086973
min magnitude class 2  =  -7.29747643241007
after normalizing >>
max magnitude class 2  =  0.9361413884937131
min magnitude class 2  =  -0.811393602518538

 cls 3  >> 
max magnitude class 3  =  2.921229641073384
min magnitude class 3  =  -8.077613775922405
after normalizing >>
max magnitude class 3  =  0.9677559433962111
min magnitude class 3  =  -0.9472210732844096

 cls 4  >> 
max magnitude class 4  =  2.566649466280018
min magnitude class 4  =  -6.7000715338116725
after normalizing >>
max magnitude class 4  =  0.9060210082256042
min magnitude class 4  =  -0.707381152563975

 cls 5  >> 
max magnitude class 5  =  3.059927687644744
min magnitude class 5  =  -7.133883777493334
after normalizing >>
max magnitude class 5 

In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -1.8343638646323206
max tst = 1.2596009116229263


In [14]:
len(wndws_test)

308

In [15]:
np.shape(wndws_test)

(308, 481)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,

<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(18, 481)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (155, 480)
y_train =>  (155,)
X_test  =>  (308, 480)
y_test  =>  (308,)
X_valid  =>  (155, 480)
y_valid  =>  (155,)


In [22]:
y_train=np.array(y_train+1)
y_test=np.array(y_test+1)
y_valid=np.array(y_valid+1)

In [23]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [24]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
2/2 [==============================] - 4s 540ms/step - loss: 5.3816 - accuracy: 0.1161 - val_loss: 1.8942 - val_accuracy: 0.2194
Epoch 2/1000
2/2 [==============================] - 0s 50ms/step - loss: 2.1629 - accuracy: 0.2323 - val_loss: 1.9206 - val_accuracy: 0.2323
Epoch 3/1000
2/2 [==============================] - 0s 44ms/step - loss: 1.7480 - accuracy: 0.2258 - val_loss: 1.8999 - val_accuracy: 0.2129
Epoch 4/1000
2/2 [==============================] - 0s 45ms/step - loss: 1.6423 - accuracy: 0.4194 - val_loss: 1.8681 - val_accuracy: 0.2194
Epoch 5/1000
2/2 [==============================] - 0s 46ms/step - loss: 1.8303 - accuracy: 0.2903 - val_loss: 1.8949 - val_accuracy: 0.2323
Epoch 6/1000
2/2 [==============================] - 0s 49ms/step - loss: 1.7231 - accuracy: 0.3484 - val_loss: 1.8665 - val_accuracy: 0.2839
Epoch 7/1000
2/2 [==============================] - 0s 50ms/step - loss: 1.6402 - accuracy: 0.3290 - val_loss: 1.8561 - val_accuracy: 0.3806
Epoch 8/1000

In [25]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [26]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

10/10 [==============================] - 0s 20ms/step
array([[ 0,  0, 28, 12, 20],
       [ 0,  0, 36, 17,  5],
       [ 0,  0, 43, 10,  6],
       [ 0,  0, 27, 19, 18],
       [ 0,  0, 16, 13, 38]], dtype=int64)
              precision    recall  f1-score   support

           2       0.00      0.00      0.00        60
           3       0.00      0.00      0.00        58
           4       0.29      0.73      0.41        59
           5       0.27      0.30      0.28        64
           6       0.44      0.57      0.49        67

    accuracy                           0.32       308
   macro avg       0.20      0.32      0.24       308
weighted avg       0.21      0.32      0.24       308



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=256, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=64, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=64, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=1024, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
2/2 [==============================] - 2s 723ms/step - loss: 17.3443 - accuracy: 0.0903 - val_loss: 1.9228 - val_accuracy: 0.2194
Epoch 2/1000
2/2 [==============================] - 0s 54ms/step - loss: 2.6458 - accuracy: 0.2387 - val_loss: 1.9169 - val_accuracy: 0.2194
Epoch 3/1000
2/2 [==============================] - 0s 60ms/step - loss: 1.7759 - accuracy: 0.1935 - val_loss: 1.8922 - val_accuracy: 0.2194
Epoch 4/1000
2/2 [==============================] - 0s 60ms/step - loss: 1.9187 - accuracy: 0.3290 - val_loss: 1.8770 - val_accuracy: 0.2194
Epoch 5/1000
2/2 [==============================] - 0s 78ms/step - loss: 1.6958 - accuracy: 0.2323 - val_loss: 1.8528 - val_accuracy: 0.2645
Epoch 6/1000
2/2 [==============================] - 0s 57ms/step - loss: 1.5532 - accuracy: 0.2710 - val_loss: 1.8455 - val_accuracy: 0.2129
Epoch 7/1000
2/2 [==============================] - 0s 52ms/step - loss: 1.6620 - accuracy: 0.2645 - val_loss: 1.8501 - val_accuracy: 0.2194
Epoch 8/100

Epoch 59/1000
2/2 [==============================] - 0s 54ms/step - loss: 0.0177 - accuracy: 1.0000 - val_loss: 2.8614 - val_accuracy: 0.1097
Epoch 60/1000
2/2 [==============================] - 0s 54ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 3.5683 - val_accuracy: 0.1097
Epoch 61/1000
2/2 [==============================] - 0s 55ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 4.2751 - val_accuracy: 0.1161
Epoch 62/1000
2/2 [==============================] - 0s 53ms/step - loss: 4.4474e-04 - accuracy: 1.0000 - val_loss: 5.0241 - val_accuracy: 0.1161
Epoch 63/1000
2/2 [==============================] - 0s 54ms/step - loss: 2.7089e-04 - accuracy: 1.0000 - val_loss: 5.7304 - val_accuracy: 0.1161
Epoch 64/1000
2/2 [==============================] - 0s 54ms/step - loss: 1.6530e-04 - accuracy: 1.0000 - val_loss: 5.8704 - val_accuracy: 0.1161
Epoch 65/1000
2/2 [==============================] - 0s 53ms/step - loss: 1.1409e-04 - accuracy: 1.0000 - val_loss: 6.3469 - val_accurac

In [28]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

10/10 [==============================] - 0s 17ms/step
array([[ 0,  0, 30, 30,  0],
       [ 0,  0, 27, 31,  0],
       [ 0,  0, 43, 16,  0],
       [ 0,  0, 25, 39,  0],
       [ 0,  0, 21, 46,  0]], dtype=int64)
              precision    recall  f1-score   support

           2       0.00      0.00      0.00        60
           3       0.00      0.00      0.00        58
           4       0.29      0.73      0.42        59
           5       0.24      0.61      0.35        64
           6       0.00      0.00      0.00        67

    accuracy                           0.27       308
   macro avg       0.11      0.27      0.15       308
weighted avg       0.11      0.27      0.15       308



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
